<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/striking_distance_creator/striking_distance_creator_(gsc_version).ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Script by @LeeFootSEO
**Early Access To Apps & Feature Requests:** https://www.patreon.com/leefootseo

**Buy me a Coffee:** https://www.buymeacoffee.com/leefootseo

# Striking Distance Report Creator v1 (GSC Version)
by [LeeFootSEO](https://twitter.com/LeeFootSEO) October 2021

In [ ]:
!pip install pandas
!pip install git+https://github.com/joshcarty/google-searchconsole
!pip install fabric3

  Cloning https://github.com/joshcarty/google-searchconsole to /tmp/pip-req-build-nk_m_hgg
  Running command git clone -q https://github.com/joshcarty/google-searchconsole /tmp/pip-req-build-nk_m_hgg
  Created wheel for searchconsole: filename=searchconsole-0.0.3-cp37-none-any.whl size=8799 sha256=dcbc31f01e5c09e92a07a058810299a7fc01fc6da48cabd9f734a3ca2488164e
  Stored in directory: /tmp/pip-ephem-wheel-cache-gxrnaxa0/wheels/eb/d4/d2/b374d767a5df3b100a2302f2dadcd170a487d48abe626c3f09
Successfully built searchconsole
     |████████████████████████████████| 102kB 4.3MB/s 
     |████████████████████████████████| 215kB 17.8MB/s 
     |████████████████████████████████| 3.2MB 17.2MB/s 
     |████████████████████████████████| 962kB 31.2MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 


In [ ]:
import re
import pandas as pd
import numpy as np
import sys
import searchconsole  # pip install git+https://github.com/joshcarty/google-searchconsole
from collections import OrderedDict
from fabric.colors import blue, green, yellow, red  # pip install fabric3
import datetime
import time

In [ ]:
# start timing the script
startTime = time.time()

In [ ]:
# Set the ctr average for positions 1-3 for forecasting.
ctr_avg = 0.209 # (20.9% is the average for blue links from 2021 sistrix study)
pagination_filters = "filterby|page|p="

In [ ]:
# Set GSC Variables Here:
start_date = "2021-01-01"  # Example formatting: "2020-09-20"
end_date = "2021-03-31"
dimension_1 = "query"  # get keyword data from Search Console
dimension_2 = "page"  # get URL data from Search Console
dimension_3 = "country"  # get country data from Search Console
dimension_4 = "device"  # Get device data from Search Console
country_filter = "gbr"  # set the country filter

In [ ]:
# Set Screaming Frog Variables Here:
url_filter = "browse"  # default = "browse" (Can also be left black to include all pages from crawl) e.g. ""
min_imps = 100  # default = 100
max_page = 20  # default = 20 results

In [ ]:
# format the date for readability to print to terminal
start_date_formatted = start_date
start_date_formatted = datetime.datetime.strptime(start_date_formatted, '%Y-%m-%d')
start_date_formatted = start_date_formatted.strftime("%d/%b/%Y")
end_date_formatted = end_date
end_date_formatted = datetime.datetime.strptime(end_date_formatted, '%Y-%m-%d')
end_date_formatted = end_date_formatted.strftime("%d/%b/%Y")

In [ ]:
# print selected variables to terminal
print(red(("URL Filter Applied: ")) + blue(url_filter))
print(red(("Minimum Impressions: ")) + blue(min_imps))
print(red(("Striking Cutoff: ")) + blue(max_page))
print(red(("Country: ")) + blue(country_filter))
print(red(("Start Date: ")) + blue(start_date_formatted))
print(red(("End Date:   ")) + blue(end_date_formatted))

URL Filter Applied: browse
Minimum Impressions: 100
Striking Cutoff: 20
Country: gbr
Start Date: 01/Jan/2021
End Date:   31/Mar/2021


In [ ]:
# import bad word list (if it exists)
try:
    df_bad_words = pd.read_csv("/python_scripts/universal_striking_distance_sc_api/bad_words/bad_kws.csv",
                                 names=["Bad KWs"], header=None)

    # convert to lower case for matching
    df_bad_words["Bad KWs"] = df_bad_words["Bad KWs"].str.lower()

    # convert to list
    bad_word_list = df_bad_words["Bad KWs"].to_list()
except FileNotFoundError:
    pass

In [ ]:
# start  ::: import screaming frog custom crawl file (must be named internal_html.csv) :::
df_sf = pd.read_csv("/python_scripts/universal_striking_distance_sc_api/place_crawl_file_here/internal_html.csv", encoding="utf8")

In [ ]:
# drop non-indexable pages from crawl file
df_sf = df_sf[~df_sf["Indexability"].isin(["Non-Indexable"])]

In [ ]:
# extracts the root domain from the crawl to match to a search console property automatically.
extracted_domain = df_sf["Address"]
url = extracted_domain.iloc[0]
print(red(("Domain is: ")) + blue(url))

In [ ]:
# start ::: authenticate with search console api
try:
    print("\nReading client_secrets.json & credentials.json..")  # Authenticate with GSC
    account = searchconsole.authenticate(
        client_config="/python_scripts/universal_striking_distance_sc_api/auth_files/client_secrets.json",
        credentials="/python_scripts/universal_striking_distance_sc_api/auth_files/credentials.json",
    )
    webproperty = account[(url)]  # connect to the gsc property
    print("Pulling GSC Data. Please Be Patient!")
    gsc_data = (
        webproperty.query.range(start_date, end_date)
        .dimension(
            dimension_1,  # set date range & dimensions
            dimension_2,
            dimension_3,
            #dimension_4,
        )
        .get()
    )
except AttributeError:
    print(red("No Data Received from Search Console API!"))
    sys.exit(1)

In [ ]:
# make the dataframe
df_gsc_data = pd.DataFrame(gsc_data)

In [ ]:
# keep only gbr traffic
df_gsc_data = df_gsc_data[df_gsc_data["country"].str.contains(country_filter)]

In [ ]:
# rename columns to match old script
df_gsc_data.rename(
    columns={
        "query": "Keyword",
        "impressions": "Volume",
        "page": "URL",
        "position": "Position",
        "clicks": "Clicks",
    },
    inplace=True,
)

In [ ]:
# start  ::: cleanup screaming frog import and rename columns :::
df_sf.rename(
    columns={
        "Title 1": "Title",
        "H1-1": "H1",
        "Canonical Link Element 1": "Canonical URL",
        "Meta Description 1": "Meta Description",
        "Category Description 1": "Category Description",
    },
    inplace=True,
)

In [ ]:
# extract missing category descriptions from sf crawl
df_sf["Category Description"] = df_sf["Category Description"].fillna("True")
df_sf_cat_desc = df_sf[["Address", "Category Description"]].copy()
df_sf_cat_desc = df_sf_cat_desc[df_sf_cat_desc["Category Description"].isin(["True"])]
df_sf_cat_desc.rename(columns={"Category Description": "Missing Copy?"}, inplace=True)

In [ ]:
# make new dataframe to lookup keyword search volume (amsv)
df_kw_pos = df_gsc_data[["Keyword", "Volume", "Clicks", "Position"]]

# make new dataframe to check which words were dropped
df_gsc_data_pre_filter = df_gsc_data[["Keyword"]].copy()

In [ ]:
# match and drop blacklisted words
try:
    df_gsc_data = df_gsc_data[~df_gsc_data.Keyword.str.contains("|".join(bad_word_list))]
    df_gsc_data_post_filter = df_gsc_data[["Keyword"]].copy()
except NameError:
    pass

In [ ]:
# make sets from pre and port filtered dfs to show which keywords were blocked
gsc_pre_filter_list = df_gsc_data_pre_filter["Keyword"].tolist()
gsc_post_filter_list = df_gsc_data_post_filter["Keyword"].tolist()

In [ ]:
# show / export blocked keywords
blocked = list(set(gsc_pre_filter_list) - set(gsc_post_filter_list))
blocked_len = len(blocked)

In [ ]:
# print the number of blocked keywords
print("Keywords blocked by bad_words.csv: ", blocked_len)

In [ ]:
# make dataframe for export
df_blocked = pd.DataFrame(blocked, columns=["Keyword"])

In [ ]:
# parse the bad word list and assign to a variable
regex = r"|".join(df_bad_words["Bad KWs"])

In [ ]:
# check the bad word list and extract the words (used to show which word trigger the block)
df_blocked_kws = pd.concat(
    [
        df_bad_words,
        df_blocked,
        (pd.DataFrame(df_blocked["Keyword"].str.findall(regex).to_list())),
    ],
    axis="columns",
).fillna("")

In [ ]:
# delete the helper column
del df_blocked_kws["Bad KWs"]

In [ ]:
# add prefix to columns
df_blocked_kws = df_blocked_kws.add_prefix("Matched ")

In [ ]:
# rename main column
df_blocked_kws.rename(columns={"Matched Keyword": "Blocked Keyword"}, inplace=True)

# replaces hyphenated keywords with a space instead of a "-" so they can be de-duplicated later.
df_gsc_data["Keyword"] = df_gsc_data["Keyword"].str.replace("-", " ")

# drop duplicate kws if -both- kw and url are duplicated.
df_gsc_data.drop_duplicates(subset=["URL", "Keyword"], keep="first", inplace=True)

# removes paginated and filterby by pages from the url column #
df_gsc_data = df_gsc_data[~df_gsc_data.URL.str.contains(pagination_filters)]

# drop rows if position > max_page variable
df_gsc_data.loc[df_gsc_data["Position"] >= max_page, "Over_21"] = "Over21"
df_gsc_data = df_gsc_data[~df_gsc_data["Over_21"].isin(["Over21"])]

# create a new dataframe to pull the keyword and search volume from to merge with the final dataframe.
df_search_kws_lookup = df_gsc_data

# sort data by a multiple columns
df_search_kws_lookup.sort_values(["URL", "Volume"], ascending=[True, False], inplace=True,)

In [ ]:
# remove sitelinks
df_gsc_data["Depth"] = (df_gsc_data["URL"].str.count("\/") - 3)  # # count folder depth
df_gsc_data.sort_values(["Depth", "URL"], ascending=[True, True], inplace=True,)  # sorting data by a multiple columns
df_gsc_data.drop_duplicates(subset=["Keyword"], keep="first", inplace=True)

# check min impression threshold and drops
df_gsc_data.loc[df_gsc_data["Volume"] < min_imps, "Under_100"] = "Under100"
df_gsc_data = df_gsc_data[~df_gsc_data["Under_100"].isin(["Under100"])].copy()

# checks if position 1-3 and drop
df_gsc_data.loc[df_gsc_data["Position"] <= 3, "KWs in Striking Dist."] = "No - Top 3 Ranking"

# check if keyword is in striking distance (4-20)
df_gsc_data.loc[df_gsc_data["Position"] > 3, "KWs in Striking Dist."] = "Yes - Striking Distance"

# drops rows before stats are consolidated (ensures top 3 kws in striking dist. & search volume are removed).
df_gsc_data = df_gsc_data[~df_gsc_data["KWs in Striking Dist."].isin(["No - Top 3 Ranking"])]

In [ ]:
# groups the urls (removes dupes and combines stats).
df_gsc_data = (
    df_gsc_data.groupby("URL")
    .agg(
        {
            "Volume": "sum",
            "KWs in Striking Dist.": "count",
            "Position": "median",
            "Clicks": "sum",
        }
    )
    .reset_index()
)

In [ ]:
# this code brings in the keywords and places them in adjacent rows
df_merged_all_kws = df_gsc_data.merge(
    df_search_kws_lookup.groupby("URL")["Keyword"]
    .apply(lambda x: x.reset_index(drop=True))
    .unstack()
    .reset_index()
)


In [ ]:
# sorts urls by largest number of kws in striking dist
df_merged_all_kws = df_merged_all_kws.sort_values(by="KWs in Striking Dist.", ascending=False)

In [ ]:
try:
    # imports only the columns needed back into the same dataframe (top five keyword suggestions)
    df_merged_all_kws = df_merged_all_kws[
        ["URL", "Volume", "Position", "Clicks", "KWs in Striking Dist.", 0, 1, 2, 3, 4]
    ]
except KeyError:
    print(colored("Not Enough Data to Make the Dataframe - Try Importing More from Search Console!", "red"))
    sys.exit(1)

In [ ]:
# this is the final striking distance df which should go to a separate worksheet.
df_merged_all_kws.rename(columns={
        "Volume": "Total Impressions",
        0: "KW1",
        1: "KW2",
        2: "KW3",
        3: "KW4",
        4: "KW5",
    }, inplace=True
)


In [ ]:
# reassign df_blocked_kws (sloppy work around from old code)
df_s_dist = df_merged_all_kws

# Merges Striking Distance DF with Screaming Frog DF To Merge in the Title, H1 and Category Description
df_s_dist = pd.merge(df_s_dist, df_sf, left_on="URL", right_on="Address", how="inner")

In [ ]:
# Defines the New Columns
new_striking_cols = [
    "URL",
    "Title",
    "H1",
    "Category Description",
    "Parent URL",
    "Total Impressions",
    "Clicks",
    "KWs in Striking Dist.",
    "Link from Parent Cat?",
    "KW1",
    "KW1 in Title",
    "KW1 in Copy",
    "KW1 Cat Text",
    "KW2",
    "KW2 in Title",
    "KW2 in Copy",
    "KW2 Cat Text",
    "KW2 Linked from Parent",
    "KW3",
    "KW3 in Title",
    "KW3 in Copy",
    "KW3 Cat Text",
    "KW3 Linked from Parent",
    "KW4",
    "KW4 in Title",
    "KW4 in Copy",
    "KW4 Cat Text",
    "KW4 Linked from Parent",
    "KW5",
    "KW5 in Title",
    "KW5 in Copy",
    "KW5 Cat Text",
    "KW5 Linked from Parent",
]

In [ ]:
# re-indexes columns to place them in a logical order + inserts new blank columns for kw checks.
df_s_dist = df_s_dist.reindex(columns=new_striking_cols)

# Routine to create parent url for pd.merge

In [ ]:
# creates temp column and extracts the last folder from the url column
df_s_dist["Temp"] = df_s_dist["URL"].str.split("/").str[-1]

# subtracts the last folder string from the url
df_s_dist["Parent URL"] = df_s_dist["URL"] = [
    a.replace(b, "").strip() for a, b in zip(df_s_dist["URL"], df_s_dist["Temp"])
]

# removes the last trailing slash so that the .html suffix can be applied.
df_s_dist["Parent URL"] = df_s_dist["Parent URL"].astype(str).str[:-1]

# appends .html to the new parent url
df_s_dist["Parent URL"] = df_s_dist["Parent URL"] + ".html"

# fixes the url column by adding the folder name.html back to it.
df_s_dist["URL"] = df_s_dist["URL"] + df_s_dist["Temp"]

# deletes ["temp"] helper column
del df_s_dist["Temp"]

In [ ]:
# make new dataframe just containing the url and category description for pd.merge to parent url
try:
    df_cat_text = df_sf[["Address", "Category Description"]]
except Exception:
    pass

# merges category text to parent url
try:
    df_s_dist = pd.merge(
        df_s_dist, df_cat_text, left_on="Parent URL", right_on="Address", how="left"
    )
except Exception:
    pass

In [ ]:
# drop duplicate values before merging keyword search volumes
df_kw_pos.drop_duplicates(subset="Keyword", inplace=True)

# rounds to 2 decimal places
df_kw_pos = df_kw_pos.round(0)

In [ ]:
# combines to keywords search volumes (amsv) sheet to lookup all kw volume - renames and deletes helper cols
df_s_dist = pd.merge(
    df_s_dist, df_kw_pos, left_on="KW1", right_on="Keyword", how="left"
)

df_s_dist.rename(columns={"Volume": "KW1 Imps", "Clicks_x": "Striking Dist. Clicks", "Clicks_y": "KW1 Clicks", "Position": "KW1 Position"}, inplace=True)
del df_s_dist["Keyword"]

df_s_dist = pd.merge(
    df_s_dist, df_kw_pos, left_on="KW2", right_on="Keyword", how="left"
)
df_s_dist.rename(columns={"Volume": "KW2 Imps", "Clicks": "KW2 Clicks", "Position": "KW2 Position"}, inplace=True)
del df_s_dist["Keyword"]

df_s_dist = pd.merge(
    df_s_dist, df_kw_pos, left_on="KW3", right_on="Keyword", how="left"
)
df_s_dist.rename(columns={"Volume": "KW3 Imps", "Clicks": "KW3 Clicks", "Position": "KW3 Position"}, inplace=True)
del df_s_dist["Keyword"]

df_s_dist = pd.merge(
    df_s_dist, df_kw_pos, left_on="KW4", right_on="Keyword", how="left"
)
df_s_dist.rename(columns={"Volume": "KW4 Imps", "Clicks": "KW4 Clicks", "Position": "KW4 Position"}, inplace=True)
del df_s_dist["Keyword"]

df_s_dist = pd.merge(
    df_s_dist, df_kw_pos, left_on="KW5", right_on="Keyword", how="left"
)
df_s_dist.rename(columns={"Volume": "KW5 Imps", "Clicks": "KW5 Clicks", "Position": "KW5 Position"}, inplace=True)
del df_s_dist["Keyword"]

# renames category description columns after merging.
df_s_dist.rename(
    columns={"Category Description_x": "Category Description"}, inplace=True
)
df_s_dist.rename(columns={"Category Description_y": "Parent Cat Text"}, inplace=True)

In [ ]:
# defines the new columns
sort_cols = [
    "URL",
    "Title",
    "H1",
    "Category Description",
    "Parent URL",
    "Total Impressions",
    "Striking Dist. Clicks",
    "KWs in Striking Dist.",
    "KW1",
    "KW1 Imps",
    "KW1 Clicks",
    "KW1 Position",
    "KW1 in Title",
    "KW1 in Copy",
    "KW1 Cat Text",
    "KW2",
    "KW2 Imps",
    "KW2 Clicks",
    "KW2 Position",
    "KW2 in Title",
    "KW2 in Copy",
    "KW2 Cat Text",
    "KW3",
    "KW3 Imps",
    "KW3 Clicks",
    "KW3 Position",
    "KW4",
    "KW4 Imps",
    "KW4 Clicks",
    "KW4 Position",
    "KW4 in Title",
    "KW4 in Copy",
    "KW4 Cat Text",
    "KW5",
    "KW5 Imps",
    "KW5 Clicks",
    "KW5 Position",
    "KW5 in Title",
    "KW5 in Copy",
    "KW5 Cat Text",
]

In [ ]:
# re-indexes columns to place them in a logical order + inserts new blank columns for kw checks.
df_s_dist = df_s_dist.reindex(columns=sort_cols)

In [ ]:
# replaces nan values with 0"s
df_s_dist["KW2"] = df_s_dist["KW2"].fillna(0)
df_s_dist["KW3"] = df_s_dist["KW3"].fillna(0)
df_s_dist["KW4"] = df_s_dist["KW4"].fillna(0)
df_s_dist["KW5"] = df_s_dist["KW5"].fillna(0)

# drops the title, h1 and category description to lower case so kws can be matched against them.
df_s_dist["Title"] = df_s_dist["Title"].str.lower()
df_s_dist["H1"] = df_s_dist["H1"].str.lower()

In [ ]:
try:
    df_s_dist["Category Description"] = df_s_dist["Category Description"].str.lower()
except Exception:
    pass
try:
    df_s_dist["Parent Cat Text"] = df_s_dist["Parent Cat Text"].str.lower()
except Exception:
    pass

In [ ]:
# change data types
df_s_dist = df_s_dist.astype(str)  # Casts Entire DF to String
float_cols = [
    "KW1 Imps",
    "KW2 Imps",
    "KW3 Imps",
    "KW4 Imps",
    "KW5 Imps",
    "KW1 Position",
    "KW2 Position",
    "KW3 Position",
    "KW4 Position",
    "KW5 Position",
    "KWs in Striking Dist.",
    "Total Impressions",
]
int_cols = [
    "KW1 Imps",
    "KW2 Imps",
    "KW3 Imps",
    "KW4 Imps",
    "KW5 Imps",
    "KW1 Position",
    "KW2 Position",
    "KW3 Position",
    "KW4 Position",
    "KW5 Position",
    "KWs in Striking Dist.",
    "Total Impressions",
]

# change the data types to ensure the script works without errors
df_s_dist[float_cols] = df_s_dist[float_cols].astype(float)
df_s_dist = df_s_dist.fillna(0)
df_s_dist[int_cols] = df_s_dist[int_cols].astype(int)

In [ ]:
# checks whether a keyword appears in title, h1 and category description.
df_s_dist["KW1 in Title"] = df_s_dist.apply(
    lambda row: row["KW1"] in row["Title"], axis=1
)
df_s_dist["KW1 in H1"] = df_s_dist.apply(lambda row: row["KW1"] in row["H1"], axis=1)
df_s_dist["KW1 Cat Text"] = df_s_dist.apply(
    lambda row: row["KW1"] in row["Category Description"], axis=1
)

df_s_dist["KW2 in Title"] = df_s_dist.apply(
    lambda row: row["KW2"] in row["Title"], axis=1
)
df_s_dist["KW2 in H1"] = df_s_dist.apply(lambda row: row["KW2"] in row["H1"], axis=1)
df_s_dist["KW2 Cat Text"] = df_s_dist.apply(
    lambda row: row["KW2"] in row["Category Description"], axis=1
)

df_s_dist["KW3 in Title"] = df_s_dist.apply(
    lambda row: row["KW3"] in row["Title"], axis=1
)
df_s_dist["KW3 in H1"] = df_s_dist.apply(lambda row: row["KW3"] in row["H1"], axis=1)
df_s_dist["KW3 Cat Text"] = df_s_dist.apply(
    lambda row: row["KW3"] in row["Category Description"], axis=1
)

df_s_dist["KW4 in Title"] = df_s_dist.apply(
    lambda row: row["KW4"] in row["Title"], axis=1
)
df_s_dist["KW4 in H1"] = df_s_dist.apply(lambda row: row["KW4"] in row["H1"], axis=1)
df_s_dist["KW4 Cat Text"] = df_s_dist.apply(
    lambda row: row["KW4"] in row["Category Description"], axis=1
)

df_s_dist["KW5 in Title"] = df_s_dist.apply(
    lambda row: row["KW5"] in row["Title"], axis=1
)
df_s_dist["KW5 in H1"] = df_s_dist.apply(lambda row: row["KW5"] in row["H1"], axis=1)
df_s_dist["KW5 Cat Text"] = df_s_dist.apply(
    lambda row: row["KW5"] in row["Category Description"], axis=1
)

In [ ]:
# restores title and h1 columns back to title case for aesthetic reasons.
df_s_dist["Title"] = df_s_dist["Title"].str.title()
df_s_dist["H1"] = df_s_dist["H1"].str.title()

In [ ]:
# delete true / false values if there is no keyword
df_s_dist.loc[df_s_dist["KW1"] == "0", "KW1 in Title"] = ""
df_s_dist.loc[df_s_dist["KW1"] == "0", "KW1 in H1"] = ""
df_s_dist.loc[df_s_dist["KW1"] == "0", "KW1 Cat Text"] = ""
df_s_dist.loc[df_s_dist["KW1"] == "0", "KW1 Linked from Parent"] = ""
df_s_dist.loc[df_s_dist["KW2"] == "0", "KW2 in Title"] = ""
df_s_dist.loc[df_s_dist["KW2"] == "0", "KW2 in H1"] = ""
df_s_dist.loc[df_s_dist["KW2"] == "0", "KW2 Cat Text"] = ""
df_s_dist.loc[df_s_dist["KW2"] == "0", "KW2 Linked from Parent"] = ""
df_s_dist.loc[df_s_dist["KW3"] == "0", "KW3 in Title"] = ""
df_s_dist.loc[df_s_dist["KW3"] == "0", "KW3 in H1"] = ""
df_s_dist.loc[df_s_dist["KW3"] == "0", "KW3 Cat Text"] = ""
df_s_dist.loc[df_s_dist["KW3"] == "0", "KW3 Linked from Parent"] = ""
df_s_dist.loc[df_s_dist["KW4"] == "0", "KW4 in Title"] = ""
df_s_dist.loc[df_s_dist["KW4"] == "0", "KW4 in H1"] = ""
df_s_dist.loc[df_s_dist["KW4"] == "0", "KW4 Cat Text"] = ""
df_s_dist.loc[df_s_dist["KW4"] == "0", "KW4 Linked from Parent"] = ""
df_s_dist.loc[df_s_dist["KW5"] == "0", "KW5 in Title"] = ""
df_s_dist.loc[df_s_dist["KW5"] == "0", "KW5 in H1"] = ""
df_s_dist.loc[df_s_dist["KW5"] == "0", "KW5 Cat Text"] = ""
df_s_dist.loc[df_s_dist["KW5"] == "0", "KW5 Linked from Parent"] = ""

In [ ]:
# deletes the 0"s from the kw cols.
df_s_dist["KW2"].replace({"0": ""}, inplace=True)
df_s_dist["KW3"].replace({"0": ""}, inplace=True)
df_s_dist["KW4"].replace({"0": ""}, inplace=True)
df_s_dist["KW5"].replace({"0": ""}, inplace=True)

In [ ]:
# slicing striking distance and removing if all values = true - then re-merging to form final dataframe.
# main dataframe to re-merge into.
df_main = df_s_dist[
    ["URL", "Title", "H1", "Total Impressions", "KWs in Striking Dist."]
]

In [ ]:
# split down into smaller dataframes, drop if all true then re-merge.
df_kw_1 = df_s_dist[
    [
        "URL",
        "KW1",
        "KW1 Imps",
        "KW1 Clicks",
        "KW1 Position",
        "KW1 in Title",
        "KW1 in H1",
        "KW1 Cat Text",
    ]
]


In [ ]:
# drops rows if all values evaluate to true. (nothing for the user to do).
df_kw_1 = df_kw_1.drop(
    df_kw_1[
        (df_kw_1["KW1 in Title"] == True)
        & (df_kw_1["KW1 in H1"] == True)
        & (df_kw_1["KW1 Cat Text"] == True)
    ].index
)

df_kw_2 = df_s_dist[
    [
        "URL",
        "KW2",
        "KW2 Imps",
        "KW2 Clicks",
        "KW2 Position",
        "KW2 in Title",
        "KW2 in H1",
        "KW2 Cat Text",
    ]
]
# drops rows if all values evaluate to true. (nothing for the user to do).
df_kw_2 = df_kw_2.drop(
    df_kw_2[
        (df_kw_2["KW2 in Title"] == True)
        & (df_kw_2["KW2 in H1"] == True)
        & (df_kw_2["KW2 Cat Text"] == True)
    ].index
)

df_kw_3 = df_s_dist[
    [
        "URL",
        "KW3",
        "KW3 Imps",
        "KW3 Clicks",
        "KW3 Position",
        "KW3 in Title",
        "KW3 in H1",
        "KW3 Cat Text",
    ]
]
# drops rows if all values evaluate to true. (nothing for the user to do).
df_kw_3 = df_kw_3.drop(
    df_kw_3[
        (df_kw_3["KW3 in Title"] == True)
        & (df_kw_3["KW3 in H1"] == True)
        & (df_kw_3["KW3 Cat Text"] == True)
    ].index
)

df_kw_4 = df_s_dist[
    [
        "URL",
        "KW4",
        "KW4 Imps",
        "KW4 Clicks",
        "KW4 Position",
        "KW4 in Title",
        "KW4 in H1",
        "KW4 Cat Text",
    ]
]
# drops rows if all values evaluate to true. (nothing for the user to do).
df_kw_4 = df_kw_4.drop(
    df_kw_4[
        (df_kw_4["KW4 in Title"] == True)
        & (df_kw_4["KW4 in H1"] == True)
        & (df_kw_4["KW4 Cat Text"] == True)
    ].index
)

df_kw_5 = df_s_dist[
    [
        "URL",
        "KW5",
        "KW5 Imps",
        "KW5 Clicks",
        "KW5 Position",
        "KW5 in Title",
        "KW5 in H1",
        "KW5 Cat Text",
    ]
]
# drops rows if all values evaluate to true. (nothing for the user to do).
df_kw_5 = df_kw_5.drop(
    df_kw_5[
        (df_kw_5["KW5 in Title"] == True)
        & (df_kw_5["KW5 in H1"] == True)
        & (df_kw_5["KW5 Cat Text"] == True)
    ].index
)

In [ ]:
# do the merges
df_kw_merge = pd.merge(df_main, df_kw_1, on="URL", how="left")
df_kw_merge = pd.merge(df_kw_merge, df_kw_2, on="URL", how="left")
df_kw_merge = pd.merge(df_kw_merge, df_kw_3, on="URL", how="left")
df_kw_merge = pd.merge(df_kw_merge, df_kw_4, on="URL", how="left")
df_kw_merge = pd.merge(df_kw_merge, df_kw_5, on="URL", how="left")

In [ ]:
# shift left on empty cells
v = df_kw_merge.values
a = [[n] * v.shape[1] for n in range(v.shape[0])]
b = pd.isnull(v).argsort(axis=1, kind="mergesort")
new_array = v[a, b]

In [ ]:
# convert numpy array back to dataframe. # this only names the columns, does not re-index them.
df_s_dist = pd.DataFrame(
    data=new_array,
    columns=[
    "URL",
    "Title",
    "H1",
    "Total Impressions",
    "KWs in Striking Dist.",
    "KW1",
    "KW1 Imps",
    "KW1 Clicks",
    "KW1 Position",
    "KW1 in Title",
    "KW1 in H1",
    "KW1 Cat Text",
    "KW2",
    "KW2 Imps",
    "KW2 Clicks",
    "KW2 Position",
    "KW2 in Title",
    "KW2 in H1",
    "KW2 Cat Text",
    "KW3",
    "KW3 Imps",
    "KW3 Clicks",
    "KW3 Position",
    "KW3 in Title",
    "KW3 in H1",
    "KW3 Cat Text",
    "KW4",
    "KW4 Imps",
    "KW4 Clicks",
    "KW4 Position",
    "KW4 in Title",
    "KW4 in H1",
    "KW4 Cat Text",
    "KW5",
    "KW5 Imps",
    "KW5 Clicks",
    "KW5 Position",
    "KW5 in Title",
    "KW5 in H1",
    "KW5 Cat Text",
],
)

In [ ]:
# remove nan values
df_s_dist.replace({"nan": 0}, inplace=True)

In [ ]:
# fill in empty values
df_s_dist.fillna(
    {
        "KW1 Clicks": 0,
        "KW2 Clicks": 0,
        "KW3 Clicks": 0,
        "KW4 Clicks": 0,
        "KW5 Clicks": 0,
        "KW1 Position": 0,
        "KW2 Position": 0,
        "KW3 Position": 0,
        "KW4 Position": 0,
        "KW5 Position": 0,
        "KW1 Imps": 0,
        "KW2 Imps": 0,
        "KW3 Imps": 0,
        "KW4 Imps": 0,
        "KW5 Imps": 0,
    },
    inplace=True,
)

In [ ]:
# change the datatype for the final output (first to float, then to int)
df_s_dist = df_s_dist.astype(
    {
        "KW1 Imps": float,
        "KW2 Imps": float,
        "KW3 Imps": float,
        "KW4 Imps": float,
        "KW5 Imps": float,
    }
)

df_s_dist = df_s_dist.astype(
    {
        "KW1 Imps": int,
        "KW2 Imps": int,
        "KW3 Imps": int,
        "KW4 Imps": int,
        "KW5 Imps": int,
    }
)

df_s_dist = df_s_dist.astype(
    {
        "KW1 Clicks": float,
        "KW2 Clicks": float,
        "KW3 Clicks": float,
        "KW4 Clicks": float,
        "KW5 Clicks": float,
    }
)

df_s_dist = df_s_dist.astype(
    {
        "KW1 Clicks": int,
        "KW2 Clicks": int,
        "KW3 Clicks": int,
        "KW4 Clicks": int,
        "KW5 Clicks": int,
    }
)

In [ ]:
# Calculate Total Striking Distance Impressions of Remaining Keywords
df_s_dist["Total Impressions"] = (
    df_s_dist["KW1 Imps"]
    + df_s_dist["KW2 Imps"]
    + df_s_dist["KW3 Imps"]
    + df_s_dist["KW4 Imps"]
    + df_s_dist["KW5 Imps"]
)

# Calculate Total Clicks
df_s_dist["Total Clicks"] = (
    df_s_dist["KW1 Clicks"]
    + df_s_dist["KW2 Clicks"]
    + df_s_dist["KW3 Clicks"]
    + df_s_dist["KW4 Clicks"]
    + df_s_dist["KW5 Clicks"]
)

In [ ]:
# drop rows if == 0
df_s_dist = df_s_dist[~df_s_dist["KW1 Imps"].isin([0])]

In [ ]:
# assign columns for .replace
col_replace = [
    "KW2 Imps",
    "KW2 Clicks",
    "KW2 Position",
    "KW3 Imps",
    "KW3 Clicks",
    "KW3 Position",
    "KW4 Imps",
    "KW4 Clicks",
    "KW4 Position",
    "KW5 Imps",
    "KW5 Clicks",
    "KW5 Position",
]

In [ ]:
df_s_dist.rename(
    columns={
        "KW1 Cat Text": "KW1 in Copy",
        "KW2 Cat Text": "KW2 in Copy",
        "KW3 Cat Text": "KW3 in Copy",
        "KW4 Cat Text": "KW4 in Copy",
        "KW5 Cat Text": "KW5 in Copy",
    },
    inplace=True,
)

In [ ]:
# calculate estimated clicks if position 1-3
df_s_dist["Predicted Clicks (1-3)"] = df_s_dist["Total Impressions"] * ctr_avg

# round estimated clicks column
df_s_dist["Predicted Clicks (1-3)"] = df_s_dist["Predicted Clicks (1-3)"].round(0)

# calculate missed clicks by subtracting actual from estimate
df_s_dist["Missed Clicks"] = df_s_dist["Predicted Clicks (1-3)"] - df_s_dist["Total Clicks"]

In [ ]:
# This is used to keep 0 values for clicks, but remove them if there's no keyword
df_s_dist.fillna({"KW1": "DELETE-000", "KW2": "DELETE-000", "KW3": "DELETE-000", "KW4": "DELETE-000",  "KW5": "DELETE-000"}, inplace=True)
df_s_dist.loc[df_s_dist['KW1'] == "DELETE-000", ["KW1 Imps", "KW1 Clicks", "KW1 Position"]] = ""
df_s_dist.loc[df_s_dist['KW2'] == "DELETE-000", ["KW2 Imps", "KW2 Clicks", "KW2 Position"]] = ""
df_s_dist.loc[df_s_dist['KW3'] == "DELETE-000", ["KW3 Imps", "KW3 Clicks", "KW3 Position"]] = ""
df_s_dist.loc[df_s_dist['KW4'] == "DELETE-000", ["KW4 Imps", "KW4 Clicks", "KW4 Position"]] = ""
df_s_dist.loc[df_s_dist['KW5'] == "DELETE-000", ["KW5 Imps", "KW5 Clicks", "KW5 Position"]] = ""
df_s_dist.replace({"DELETE-000": ""}, inplace=True)

In [ ]:
# merge in the missing category description Data
df_s_dist = pd.merge(df_s_dist, df_sf_cat_desc, left_on="URL", right_on="Address", how="left")

In [ ]:
# populate columns with missing text as False
df_s_dist["Missing Copy?"] = df_s_dist["Missing Copy?"].fillna(False)
df_s_dist["Missing Copy?"] = df_s_dist["Missing Copy?"].astype(bool)

In [ ]:
col_sort = [
    "URL",
    "Title",
    "H1",
    "Missing Copy?",
    "Total Impressions",
    "Total Clicks",
    "Predicted Clicks (1-3)",
    "Missed Clicks",
    "KWs in Striking Dist.",
    "KW1",
    "KW1 Clicks",
    "KW1 Imps",
    "KW1 Position",
    "KW1 in Title",
    "KW1 in H1",
    "KW1 in Copy",
    "KW2",
    "KW2 Clicks",
    "KW2 Imps",
    "KW2 Position",
    "KW2 in Title",
    "KW2 in H1",
    "KW2 in Copy",
    "KW3",
    "KW3 Clicks",
    "KW3 Imps",
    "KW3 Position",
    "KW3 in Title",
    "KW3 in H1",
    "KW3 in Copy",
    "KW4",
    "KW4 Clicks",
    "KW4 Imps",
    "KW4 Position",
    "KW4 in Title",
    "KW4 in H1",
    "KW4 in Copy",
    "KW5",
    "KW5 Clicks",
    "KW5 Imps",
    "KW5 Position",
    "KW5 in Title",
    "KW5 in H1",
    "KW5 in Copy",
]

In [ ]:
# reindex the columns
df_s_dist = df_s_dist.reindex(columns=col_sort)

In [ ]:
# sort by impressions
df_s_dist = df_s_dist.sort_values(by="Total Impressions", ascending=False)

In [ ]:
# make the word fragments for testing
# change the data type for concat
df_s_dist = df_s_dist.astype({"KW1": str, "KW2": str, "KW3": str, "KW4": str, "KW5": str})

In [ ]:
# add the strings into a new column
df_s_dist["All Missing Words"] = df_s_dist["KW1"] + " " + df_s_dist["KW2"] + " " + df_s_dist["KW3"] + " " + df_s_dist["KW4"] + " " + df_s_dist["KW5"]

# removes non-consecutive duplicates from a single cell
df_s_dist["All Missing Words"] = (df_s_dist["All Missing Words"].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(" "))

# replace all nan values after concatenating columns
df_s_dist["All Missing Words"] = df_s_dist["All Missing Words"].apply(lambda x: x.replace(" nan", ""))

# change to comma separated
df_s_dist["All Missing Words"] = df_s_dist["All Missing Words"].apply(lambda x: x.replace(" ", ", "))

In [ ]:
# removes all nans
df_s_dist["KW1"] = df_s_dist["KW1"].apply(lambda x: x.replace("nan", ""))
df_s_dist["KW2"] = df_s_dist["KW2"].apply(lambda x: x.replace("nan", ""))
df_s_dist["KW3"] = df_s_dist["KW3"].apply(lambda x: x.replace("nan", ""))
df_s_dist["KW4"] = df_s_dist["KW4"].apply(lambda x: x.replace("nan", ""))
df_s_dist["KW5"] = df_s_dist["KW5"].apply(lambda x: x.replace("nan", ""))

In [ ]:
# checks if any word in a single column if found in the title column and remove it in place
df_s_dist["Missing Words in Title"] = [
    ", ".join([w for w in ws if re.search(f"\\b{w}\\b", s)])
    for s, ws in zip(
        df_s_dist["Title"].str.lower(),
        df_s_dist["All Missing Words"].str.split(",\s*"),
    )
]

In [ ]:
# this inverts the words to show the missing words, rather than the matched. # todo much better way in the future!
inter = df_s_dist["All Missing Words"].str.split(",").apply(set) - df_s_dist["Missing Words in Title"].str.split(",").apply(set).values
df_s_dist["Missing Words in Title"] = inter.str.join(",")
df_s_dist["Missing Words in Title"] = df_s_dist["Missing Words in Title"].str.strip()  # remove the whitespace

# checks if any word in a single column if found in the title column and remove it in place
df_s_dist["Missing Words in H1"] = [
    ", ".join([w for w in ws if re.search(f"\\b{w}\\b", s)])
    for s, ws in zip(
        df_s_dist["Title"].str.lower(),
        df_s_dist["All Missing Words"].str.split(",\s*"),
    )
]

# this inverts the words to show the missing words, rather than the matched. # todo much better way in the future!
inter2 = df_s_dist["All Missing Words"].str.split(",").apply(set) - df_s_dist["Missing Words in H1"].str.split(",").apply(set).values
df_s_dist["Missing Words in H1"] = inter2.str.join(",")
df_s_dist["Missing Words in H1"] = df_s_dist["Missing Words in H1"].str.strip()  # remove the whitespace

# make mini dataframe to merge in the category description for checking
df_mini_copy = df_sf[["Address", "Category Description"]].copy()

# do the merge
df_s_dist = pd.merge(df_s_dist, df_mini_copy, left_on="URL", right_on="Address", how="left")

df_s_dist["Category Description"] = df_s_dist["Category Description"].astype(str)
df_s_dist["Category Description"] = df_s_dist["Category Description"].apply(lambda x: x.replace(" ", ", "))

# checks if any word in a single column if found in the category description and removes it in place.
df_s_dist["Missing Words in Copy"] = [
    ", ".join([w for w in ws if re.search(f"\\b{w}\\b", s)])
    for s, ws in zip(
        df_s_dist["Category Description"].str.lower(),
        df_s_dist["All Missing Words"].str.split(",\s*"),
    )
]

# this inverts the words to show the missing words, rather than the matched. # todo much better way in the future!
inter3 = df_s_dist["All Missing Words"].str.split(",").apply(set) - df_s_dist["Missing Words in Copy"].str.split(",").apply(set).values
df_s_dist["Missing Words in Copy"] = inter3.str.join(",")
df_s_dist["Missing Words in Copy"] = df_s_dist["Missing Words in Copy"].str.strip()  # remove the whitespace

In [ ]:
# final cleanup
del df_s_dist["Address"]
del df_s_dist["Category Description"]

In [ ]:
# cleans up / standarises the comma separated keywords
df_s_dist["Missing Words in Title"] = df_s_dist["Missing Words in Title"].apply(lambda x: x.replace(",", ", "))
df_s_dist["Missing Words in Title"] = df_s_dist["Missing Words in Title"].apply(lambda x: x.replace(",  ", ", "))
df_s_dist["Missing Words in H1"] = df_s_dist["Missing Words in H1"].apply(lambda x: x.replace(",", ", "))
df_s_dist["Missing Words in H1"] = df_s_dist["Missing Words in H1"].apply(lambda x: x.replace(",  ", ", "))
df_s_dist["Missing Words in Copy"] = df_s_dist["Missing Words in Copy"].apply(lambda x: x.replace(",", ", "))
df_s_dist["Missing Words in Copy"] = df_s_dist["Missing Words in Copy"].apply(lambda x: x.replace(",  ", ", "))

In [ ]:
# start - make the page title df_blocked_kws
df_title = df_s_dist[
    [
        "URL",
        "Title",
        "Missing Words in Title",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in Title",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in Title",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in Title",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in Title",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in Title",
    ]
].copy()

In [ ]:
# check if value is true and then shifts cells
cols = ["KW1", "KW1 Clicks", "KW1 Imps", "KW1 Position", "KW1 in Title"]
x = df_title["KW1 in Title"] == True

df_title.loc[x, cols] = df_title.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW2", "KW2 Clicks", "KW2 Imps", "KW2 Position", "KW2 in Title"]
x = df_title["KW2 in Title"] == True

df_title.loc[x, cols] = df_title.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW3", "KW3 Clicks", "KW3 Imps", "KW3 Position", "KW3 in Title"]
x = df_title["KW3 in Title"] == True

df_title.loc[x, cols] = df_title.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW4", "KW4 Clicks", "KW4 Imps", "KW4 Position", "KW4 in Title"]
x = df_title["KW4 in Title"] == True

df_title.loc[x, cols] = df_title.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW5", "KW5 Clicks", "KW5 Imps", "KW5 Position", "KW5 in Title"]
x = df_title["KW5 in Title"] == True

df_title.loc[x, cols] = df_title.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

In [ ]:
# shift left on empty cells
v = df_title.values
a = [[n] * v.shape[1] for n in range(v.shape[0])]
b = pd.isnull(v).argsort(axis=1, kind="mergesort")
new_array = v[a, b]

In [ ]:
# make new dataframe from left shifted columns
df_title = pd.DataFrame(
    data=new_array,
    columns=[
        "URL",
        "Title",
        "Missing Words in Title",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in Title",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in Title",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in Title",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in Title",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in Title",
    ],
)

In [ ]:
# delete the helper columns
del df_title["KW1 in Title"]
del df_title["KW2 in Title"]
del df_title["KW3 in Title"]
del df_title["KW4 in Title"]
del df_title["KW5 in Title"]

In [ ]:
# replace empty strings with NaN values
df_title["KW1"] = df_title["KW1"].replace(r"^\s*$", np.nan, regex=True)

# keep only non nan values in KW1
df_title = df_title[df_title["KW1"].notna()]

# sort on URL
df_title = df_title.sort_values(by="URL", ascending=True)

In [ ]:
# start - make the page h1 df_blocked_kws
df_h1 = df_s_dist[
    [
        "URL",
        "H1",
        "Missing Words in H1",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in H1",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in H1",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in H1",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in H1",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in H1",
    ]
].copy()


In [ ]:
# Check if value is True and then shifts cells
cols = ["KW1", "KW1 Clicks", "KW1 Imps", "KW1 Position", "KW1 in H1"]
x = df_h1["KW1 in H1"] == True

df_h1.loc[x, cols] = df_h1.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW2", "KW2 Clicks", "KW2 Imps", "KW2 Position", "KW2 in H1"]
x = df_h1["KW2 in H1"] == True

df_h1.loc[x, cols] = df_h1.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW3", "KW3 Clicks", "KW3 Imps", "KW3 Position", "KW3 in H1"]
x = df_h1["KW3 in H1"] == True

df_h1.loc[x, cols] = df_h1.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW4", "KW4 Clicks", "KW4 Imps", "KW4 Position", "KW4 in H1"]
x = df_h1["KW4 in H1"] == True

df_h1.loc[x, cols] = df_h1.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW5", "KW5 Clicks", "KW5 Imps", "KW5 Position", "KW5 in H1"]
x = df_h1["KW5 in H1"] == True

df_h1.loc[x, cols] = df_h1.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

In [ ]:
# shift left on empty cells
v = df_h1.values
a = [[n] * v.shape[1] for n in range(v.shape[0])]
b = pd.isnull(v).argsort(axis=1, kind="mergesort")
new_array = v[a, b]

In [ ]:
# make new dataframe from left shifted columns
df_h1 = pd.DataFrame(
    data=new_array,
    columns=[
        "URL",
        "H1",
        "Missing Words in H1",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in H1",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in H1",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in H1",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in H1",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in H1",
    ],
)

In [ ]:
# delete the extra cols
del df_h1["KW1 in H1"]
del df_h1["KW2 in H1"]
del df_h1["KW3 in H1"]
del df_h1["KW4 in H1"]
del df_h1["KW5 in H1"]

In [ ]:
# replace empty strings with NaN values
df_h1["KW1"] = df_h1["KW1"].replace(r"^\s*$", np.nan, regex=True)

# keep only non nan values in KW1
df_h1 = df_h1[df_h1["KW1"].notna()]

# sort on URL
df_h1 = df_h1.sort_values(by="URL", ascending=True)

In [ ]:
# start - make the page copy df_blocked_kws
df_copy = df_s_dist[
    [
        "URL",
        "H1",
        "Missing Words in Copy",
        "Missing Copy?",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in Copy",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in Copy",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in Copy",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in Copy",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in Copy",
    ]
].copy()

In [ ]:
# Check if value is True and then shifts cells
cols = ["KW1", "KW1 Clicks", "KW1 Imps", "KW1 Position", "KW1 in Copy"]
x = df_copy["KW1 in Copy"] == True

df_copy.loc[x, cols] = df_copy.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW2", "KW2 Clicks", "KW2 Imps", "KW2 Position", "KW2 in Copy"]
x = df_copy["KW2 in Copy"] == True

df_copy.loc[x, cols] = df_copy.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW3", "KW3 Clicks", "KW3 Imps", "KW3 Position", "KW3 in Copy"]
x = df_copy["KW3 in Copy"] == True

df_copy.loc[x, cols] = df_copy.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW4", "KW4 Clicks", "KW4 Imps", "KW4 Position", "KW4 in Copy"]
x = df_copy["KW4 in Copy"] == True

df_copy.loc[x, cols] = df_copy.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

cols = ["KW5", "KW5 Clicks", "KW5 Imps", "KW5 Position", "KW5 in Copy"]
x = df_copy["KW5 in Copy"] == True

df_copy.loc[x, cols] = df_copy.loc[x, cols].apply(lambda x: x.shift(-5), axis=1)

In [ ]:
# shift left on empty cells
v = df_copy.values
a = [[n] * v.shape[1] for n in range(v.shape[0])]
b = pd.isnull(v).argsort(axis=1, kind="mergesort")
new_array = v[a, b]


In [ ]:
# make new dataframe from left shifted columns
df_copy = pd.DataFrame(
    data=new_array,
    columns=[
        "URL",
        "H1",
        "Missing Words in Copy",
        "Missing Copy?",
        "KW1",
        "KW1 Clicks",
        "KW1 Imps",
        "KW1 Position",
        "KW1 in Copy",
        "KW2",
        "KW2 Clicks",
        "KW2 Imps",
        "KW2 Position",
        "KW2 in Copy",
        "KW3",
        "KW3 Clicks",
        "KW3 Imps",
        "KW3 Position",
        "KW3 in Copy",
        "KW4",
        "KW4 Clicks",
        "KW4 Imps",
        "KW4 Position",
        "KW4 in Copy",
        "KW5",
        "KW5 Clicks",
        "KW5 Imps",
        "KW5 Position",
        "KW5 in Copy",
    ],
)

In [ ]:
# delete the helper cols
del df_copy["KW1 in Copy"]
del df_copy["KW2 in Copy"]
del df_copy["KW3 in Copy"]
del df_copy["KW4 in Copy"]
del df_copy["KW5 in Copy"]

In [ ]:
# replace empty strings with nan values
df_copy["KW1"] = df_copy["KW1"].replace(r"^\s*$", np.nan, regex=True)

# keep only non nan values in kw1
df_copy = df_copy[df_copy["KW1"].notna()]

# sort on URL
df_copy = df_copy.sort_values(by="URL", ascending=True)

In [ ]:
# make new dataframe for missing copy
df_copy_missing = df_copy.copy()

# delete rows that contain true or false
df_copy = df_copy[df_copy["Missing Copy?"].isin([False])]
df_copy_missing = df_copy_missing[df_copy_missing["Missing Copy?"].isin([True])]

del df_copy["Missing Copy?"]
del df_copy_missing["Missing Copy?"]


In [ ]:
# start ::: final output :::
print("Exporting Data to Excel.")

# format variables to insert into the filename
start_date_formatted = start_date_formatted.replace('/', '-')
end_date_formatted = end_date_formatted.replace('/', '-')
url = url.replace('https://', '')
url = url.replace('http://', '')
url = url.replace('www.', '')
url = url.replace('/', '')

In [ ]:
# create a pandas excel writer using xlsxwriter as the engine.
writer = pd.ExcelWriter(
    "/python_scripts/universal_striking_distance_sc_api/"
    + "Striking Distance Report ("
    + str(start_date_formatted)
    + " - "
    + str(end_date_formatted)
    + ") - "
    + str(url)
    + ".xlsx",
    engine="xlsxwriter",
)

In [ ]:
# convert the dataframes to xlsxwriter excel objects
df_s_dist.to_excel(writer, sheet_name="Striking Distance KWs", index=False)
df_title.to_excel(writer, sheet_name="Titles - Striking KWs", index=False)
df_h1.to_excel(writer, sheet_name="H1 - Striking KWs", index=False)
df_copy.to_excel(writer, sheet_name="Copy - Striking KWs", index=False)
df_copy_missing.to_excel(writer, sheet_name="Copy - Missing", index=False)
df_blocked_kws.to_excel(writer, sheet_name="Blocked KW List", index=False)

In [ ]:
# freeze the top rows
writer.sheets["Striking Distance KWs"].freeze_panes(1, 0)
writer.sheets["Titles - Striking KWs"].freeze_panes(1, 0)
writer.sheets["H1 - Striking KWs"].freeze_panes(1, 0)
writer.sheets["Copy - Striking KWs"].freeze_panes(1, 0)
writer.sheets["Copy - Missing"].freeze_panes(1, 0)
writer.sheets["Blocked KW List"].freeze_panes(1, 0)

In [ ]:
# start ::: apply excel formatting :::
print("Applying Excel Formatting.")

# get xlsx writer workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets["Striking Distance KWs"]

# set formatting styles
format_num_cent = workbook.add_format({"num_format": "#,##0", "align": "centre"})

# centre alignment
format_cent = workbook.add_format({"align": "centre"})

# bold text
bold = workbook.add_format({"bold": True})

# set true / false to red or green
false_format = workbook.add_format({"bg_color": "#FFC7CE", "font_color": "#9C0006"})  # Used to Colour False Cells Red
true_format = workbook.add_format({"bg_color": "#C6EFCE", "font_color": "#006100"})  # Used to Colour True Cells Green


In [ ]:
# set formatting style: formats text that contains false to red
worksheet.conditional_format(
    "A1:XFD1048576",
    {
        "type": "text",
        "criteria": "containing",
        "value": "FALSE",
        "format": false_format,
    },
)
# set formatting style: formats text that contains true to green
worksheet.conditional_format(
    "A1:XFD1048576",
    {"type": "text", "criteria": "containing", "value": "TRUE", "format": true_format},
)


In [ ]:
# set style and parameters to format worksheets as tables
header_cell_format = workbook.add_format()
header_cell_format.set_rotation(90)
header_cell_format.set_align("center")
header_cell_format.set_align("vcenter")
my_style = "Table Style Medium 7"

In [ ]:
# format sheets as tables - main df
col_names = [{"header": col_name} for col_name in df_s_dist.columns]
worksheet.add_table(
    0,
    0,
    df_s_dist.shape[0],
    df_s_dist.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

col_names = [{"header": col_name} for col_name in df_title.columns]
writer.sheets["Titles - Striking KWs"].add_table(
    0,
    0,
    df_title.shape[0],
    df_title.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

col_names = [{"header": col_name} for col_name in df_h1.columns]
writer.sheets["H1 - Striking KWs"].add_table(
    0,
    0,
    df_h1.shape[0],
    df_h1.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

col_names = [{"header": col_name} for col_name in df_copy.columns]
writer.sheets["Copy - Striking KWs"].add_table(
    0,
    0,
    df_copy.shape[0],
    df_copy.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

col_names = [{"header": col_name} for col_name in df_copy_missing.columns]
writer.sheets["Copy - Missing"].add_table(
    0,
    0,
    df_copy_missing.shape[0],
    df_copy_missing.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

col_names = [{"header": col_name} for col_name in df_blocked_kws.columns]
writer.sheets["Blocked KW List"].add_table(
    0,
    0,
    df_blocked_kws.shape[0],
    df_blocked_kws.shape[1] - 1,
    {"columns": col_names, "style": my_style},
)

In [ ]:
# apply color scales
writer.sheets["Striking Distance KWs"].conditional_format(
    "E2:E200000",
    {
        "type": "3_color_scale",
        "min_color": "#F8696B",
        "mid_color": "#FFEB84",
        "max_color": "#63BE7B",
    },
)
writer.sheets["Striking Distance KWs"].conditional_format(
    "F2:F200000",
    {
        "type": "3_color_scale",
        "min_color": "#F8696B",
        "mid_color": "#FFEB84",
        "max_color": "#63BE7B",
    },
)


In [ ]:
# set the col width, centre and format as an int with ","
worksheet.set_column("D:D", 18, format_num_cent)
worksheet.set_column("E:E", 18, format_num_cent)
worksheet.set_column("F:F", 18, format_num_cent)
worksheet.set_column("G:G", 18, format_num_cent)
worksheet.set_column("H:H", 18, format_num_cent)
worksheet.set_column("I:I", 18, format_num_cent)
worksheet.set_column("K:K", 18, format_num_cent)
worksheet.set_column("L:L", 18, format_num_cent)
worksheet.set_column("M:M", 18, format_num_cent)
worksheet.set_column("S:S", 18, format_num_cent)
worksheet.set_column("T:T", 18, format_num_cent)
worksheet.set_column("Y:Y", 18, format_num_cent)
worksheet.set_column("Z:Z", 18, format_num_cent)
worksheet.set_column("R:R", 18, format_num_cent)
worksheet.set_column("AA:AA", 18, format_num_cent)
worksheet.set_column("AF:AF", 18, format_num_cent)
worksheet.set_column("AG:AG", 18, format_num_cent)
worksheet.set_column("AH:AH", 18, format_num_cent)
worksheet.set_column("AM:AM", 18, format_num_cent)
worksheet.set_column("AN:AN", 18, format_num_cent)
worksheet.set_column("AO:AO", 18, format_num_cent)

In [ ]:
# format col widths only
worksheet.set_column("A:A", 80)  # Set URL Col Width
worksheet.set_column("B:B", 50)  # Set Page Title Col Width
worksheet.set_column("C:C", 35)  # Set H1 Col Width
worksheet.set_column("J:J", 25)  # Set KW1 Col Width
worksheet.set_column("Q:Q", 25)  # Set KW2 Col Width
worksheet.set_column("X:X", 25)  # Set KW3 Col Width
worksheet.set_column("AE:AE", 25)  # Set KW3 Col Width
worksheet.set_column("AL:AL", 25)  # Set KW4 Col Width
worksheet.set_column("AS:AS", 25)  # Set KW4 Col Width
worksheet.set_column("AT:AT", 25)  # Set KW4 Col Width
worksheet.set_column("AU:AU", 25)  # Set KW4 Col Width
worksheet.set_column("AV:AV", 25)  # Set KW4 Col Width

In [ ]:
# format the urls columns for the additional sheets
writer.sheets["Titles - Striking KWs"].set_column("A:A", 80)  # Set URL Col Width
writer.sheets["H1 - Striking KWs"].set_column("A:A", 80)  # Set URL Col Width
writer.sheets["Copy - Striking KWs"].set_column("A:A", 80)  # Set URL Col Width
writer.sheets["Copy - Missing"].set_column("A:A", 80)  # Set URL Col Width
writer.sheets["Blocked KW List"].set_column("A:A", 80)  # Set Blocked Keyword Column

In [ ]:
# format the page title column
writer.sheets["Titles - Striking KWs"].set_column("B:B", 50)  # Set Page Title Col Width

In [ ]:
# format the h1 column
writer.sheets["H1 - Striking KWs"].set_column("B:B", 35)  # Set h1 Col Width
writer.sheets["Copy - Striking KWs"].set_column("B:B", 35)  # Set h1 Col Width
writer.sheets["Copy - Missing"].set_column("B:B", 35)  # Set h1 Col Width

In [ ]:
# format the keyword columns
writer.sheets["Titles - Striking KWs"].set_column("C:C", 25)  # Set Keyword Col Width
writer.sheets["Titles - Striking KWs"].set_column("D:D", 25)  # Set Keyword Col Width
writer.sheets["Titles - Striking KWs"].set_column("H:H", 25)  # Set Keyword Col Width
writer.sheets["Titles - Striking KWs"].set_column("L:L", 25)  # Set Keyword Col Width
writer.sheets["Titles - Striking KWs"].set_column("P:P", 25)  # Set Keyword Col Width
writer.sheets["Titles - Striking KWs"].set_column("T:T", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("C:C", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("D:D", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("H:H", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("L:L", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("P:P", 25)  # Set Keyword Col Width
writer.sheets["H1 - Striking KWs"].set_column("T:T", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("C:C", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("D:D", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("H:H", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("L:L", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("P:P", 25)  # Set Keyword Col Width
writer.sheets["Copy - Striking KWs"].set_column("T:T", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("C:C", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("D:D", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("H:H", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("L:L", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("P:P", 25)  # Set Keyword Col Width
writer.sheets["Copy - Missing"].set_column("T:T", 25)  # Set Keyword Col Width
writer.sheets["Blocked KW List"].set_column("B:B", 25)  # Set Keyword Col Width
writer.sheets["Blocked KW List"].set_column("C:C", 25)  # Set Keyword Col Width
writer.sheets["Blocked KW List"].set_column("D:D", 25)  # Set Keyword Col Width

In [ ]:
# format and center all numbers
writer.sheets["Titles - Striking KWs"].set_column("E:E", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("F:F", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("G:G", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("I:I", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("J:J", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("K:K", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("M:M", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("N:N", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("O:O", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("Q:Q", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("R:R", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("S:S", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("U:U", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("V:V", 18, format_num_cent)
writer.sheets["Titles - Striking KWs"].set_column("W:W", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("E:E", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("F:F", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("G:G", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("I:I", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("J:J", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("K:K", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("M:M", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("N:N", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("O:O", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("Q:Q", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("R:R", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("S:S", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("U:U", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("V:V", 18, format_num_cent)
writer.sheets["H1 - Striking KWs"].set_column("W:W", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("E:E", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("F:F", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("G:G", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("I:I", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("J:J", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("K:K", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("M:M", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("N:N", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("O:O", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("Q:Q", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("R:R", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("S:S", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("U:U", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("V:V", 18, format_num_cent)
writer.sheets["Copy - Striking KWs"].set_column("W:W", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("E:E", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("F:F", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("G:G", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("I:I", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("J:J", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("K:K", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("M:M", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("N:N", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("O:O", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("Q:Q", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("R:R", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("S:S", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("U:U", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("V:V", 18, format_num_cent)
writer.sheets["Copy - Missing"].set_column("W:W", 18, format_num_cent)

In [ ]:
# close the pandas excel writer and output the excel file.
writer.save()

print("The script took {0} seconds!".format(time.time() - startTime))